In [5]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from supabase import create_client

# 1. 환경변수 불러오기
load_dotenv()
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_SERVICE_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

# 2. Supabase & 임베딩 & 벡터스토어 설정
supabase_client = create_client(supabase_url, supabase_key)
embedding = OpenAIEmbeddings(openai_api_key=openai_key)
vectorstore = SupabaseVectorStore(
    client=supabase_client,
    embedding=embedding,
    table_name="messages",
    query_name="match_documents"
)

# 3. 기업명-전화번호 딕셔너리 정의
CUSTOMER_SERVICE_NUMBERS = {
    "질병관리청": "1339", "국민건강보험공단": "1577-1000", "국세청": "126",
    "KB국민은행": "1588-9999", "신한은행": "1577-8000", "우리은행": "1588-5000",
    "삼성카드": "1588-8700", "KT": "100", "카카오": "1577-3754", "네이버": "1588-3819",
    "쿠팡": "1577-7011", "배달의민족": "1600-0987", "CJ대한통운": "1588-1255"
    # 필요시 계속 추가
}

# 4. 고객센터 번호 매핑 함수
def get_customer_number(text):
    for org, phone in CUSTOMER_SERVICE_NUMBERS.items():
        if org in text:
            return phone
    return None

# 5. CSV 불러오기 (사용자 파일명으로 교체)
df = pd.read_csv("datas.csv")  # ← 여기에 실제 파일명 입력
docs = []

for _, row in df.iterrows():
    text = str(row["text"])
    label = int(row["label"])
    phone = get_customer_number(text)

    metadata = {"label": label}
    if phone:
        metadata["전화번호"] = phone

    docs.append(Document(page_content=text, metadata=metadata))

# 6. Supabase에 벡터 저장
vectorstore.add_documents(docs)
print(f"✅ 총 {len(docs)}건의 문서가 Supabase에 저장되었습니다.")


ModuleNotFoundError: No module named 'langchain_community'